In [159]:
# import
import keras
import sys
import numpy as np
import string
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense
from keras.utils import np_utils, plot_model
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn import model_selection
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, precision_recall_fscore_support
import h5py as h5py

In [160]:
BINARY = False
timestep = 1
epochs = 50
en_corpus_file = "corpus-en.txt"
ewo_corpus_file = "corpus-ewo.txt"
best_model_file = "best-model-conll.hdfs"
max_nb_of_phrases =  -1
duplication = 1
max_depth = 0
is_only_vocab = True
shuffle = is_only_vocab
h1_size = 640
h2_size = 160

In [161]:
def getTag(aString):
    tag = "O"
    if BINARY:
        if aString != "O":
            return "NE"
    else:
        tag = aString
    return tag
     

In [162]:
def load_corpus(file, max_nb_of_phrases):
    nb_of_phrases = 0
    dataset = {"word": [], "ne-tag": []}
    with open(file) as f:
        prev_line = None
        for cpt, line in enumerate(f):
            if cpt == 0:
                continue
            if nb_of_phrases == max_nb_of_phrases:
                break;

            l = line.strip()
            if len(l) == 0 and len(prev_line) != 0:
                nb_of_phrases += 1
                dataset["word"].append(line)
                dataset["ne-tag"].append(None)
            else:
                l = l.split("\t")
                if l[0] not in string.punctuation:
                    dataset["word"].append(l[0])
                    dataset["ne-tag"].append(ne_type(l[1]))
            prev_line = line.strip()
        
    return pd.DataFrame(dataset), nb_of_phrases+1

In [163]:
def corpus_fingerprint(aDataframe, nb_of_biphrases):
    # create distributionnal signature
    fingerprints = {}
    current_bi_phrase_index = 0
    nb_word_in_corpus = aDataframe[aDataframe.word != "\n"].word.size
    words_in_current_phrase = []
    for index, row in aDataframe.iterrows():
        if current_bi_phrase_index > nb_of_biphrases:
            break
            
        word = row['word']
        
        if word != "\n":
            words_in_current_phrase.append(word)
            if word not in fingerprints:
                fingerprints[word] = np.zeros(nb_of_biphrases, dtype=np.float32)
            fingerprints[word][current_bi_phrase_index] += 1
        else:
            nb_word_in_current_phrase = len(words_in_current_phrase)
#             for w in words_in_current_phrase:
#                 fingerprints[w][current_bi_phrase_index] = nb_word_in_corpus / fingerprints[w][current_bi_phrase_index]                
            current_bi_phrase_index += 1
            words_in_current_phrase = []
    for word in fingerprints:
        for i in range(nb_of_biphrases):
            if fingerprints[word][i] != 0:
                fingerprints[word][i] = nb_word_in_corpus / fingerprints[word][i]
#         fingerprints[word][nb_of_biphrases] = nb_word_in_corpus / aDataframe[aDataframe.word == word].word.size
        
    return pd.DataFrame(fingerprints)

In [164]:
def corpus2trainingdata(aDataframe, fingerprintsDataFrame):
    X = np.zeros((aDataframe.shape[0], fingerprintsDataFrame.shape[0]), dtype=np.int8)
    y = np.zeros(aDataframe.shape[0], dtype=np.int8)
    i = 0
    for row in aDataframe.iterrows():
        X[i] = fingerprintsDataFrame[row[1]['word']].values
        y[i] = tag2int[getTag(row[1]['ne-tag'])]
        i += 1
    return X, y

In [165]:
def train_test_split(X, y, test_size = 0.33):
    total = X.shape[0]
    train_length = round(total * (1 - test_size)) 
    return X[:train_length], X[train_length:], y[:train_length], y[train_length:]

In [166]:
  
def ne_type(aType):
    aType = aType.lower()
    if 'per' in aType:
        t =  'NE' if BINARY else 'PER' 
    elif 'loc' in aType:
        t =  'NE' if BINARY else 'LOC'
    elif 'org' in aType:
        t =  'NE' if BINARY else 'ORG'
    elif 'hour' in aType:
        t =  'NE' if BINARY else 'MISC'
    elif aType != 'o' and len(aType) > 0 :
        t =  'NE' if BINARY else 'MISC'
    else:
        t = 'O'
    return t

In [167]:
def compute_performance(y_true, y_pred, words=None, BINARY=False):
    if BINARY:
        p = precision_score(y_true, y_pred, pos_label=tag2int['NE'])
        r = recall_score(y_true, y_pred, pos_label=tag2int['NE'])
        f1 = f1_score(y_true, y_pred, pos_label=tag2int['NE'])
        acc = accuracy_score(y_true, y_pred)
    else:
        p = precision_score(y_pred, y_true, average='macro')
        r = recall_score(y_pred, y_true, average='macro')
        f1 = f1_score(y_pred, y_true, average='macro')
        acc = accuracy_score(y_pred, y_true)
    if words is None:
        model_output_vs = pd.DataFrame({'y_true': [int2tag[i] for i in y_true], 'y_pred': [int2tag[i] for i in y_pred]})
    else:
        model_output_vs = pd.DataFrame({'word': words, 'y_true': [int2tag[i] for i in y_true], 'y_pred': [int2tag[i] for i in y_pred]})

    return p, r, f1, acc, model_output_vs

In [168]:
def P_R_F1(y_pred, y_true, neg_class):
    same = y_pred[y_true==y_pred]
    tp = same[same != neg_class].size
    nb_of_pos_exple = y_true[y_true != neg_class].size
    nb_of_pos_pred = y_pred[y_pred != neg_class].size
    p = r = f1 = 0
    try:
        p = np.round(tp*100/nb_of_pos_pred, 2)
    except ZeroDivisionError:
        print("number of correct positive predictions is 0")
        
    try:
        r = np.round(tp*100/nb_of_pos_exple, 2)
    except ZeroDivisionError:
        print("number of position exple is 0")
        
    try:
        f1 = np.round(2*r*p/(r+p), 2)
    except ZeroDivisionError:
        print("Recall and precision are 0")

    return p, r, f1

In [169]:
def is_mergeable(aListOfConsecutiveTokens, corpus, fingerprints):
    n = len(aListOfConsecutiveTokens)
    if n <= 1:
        return False
    if n == 2:
        w1, w2 = aListOfConsecutiveTokens[0], aListOfConsecutiveTokens[1]
        rep1, rep2 = fingerprints[aListOfConsecutiveTokens[0]], fingerprints[aListOfConsecutiveTokens[1]]
        tag1, tag2 = corpus[corpus.word==w1].iloc[0]['ne-tag'], corpus[corpus.word==w2].iloc[0]['ne-tag']
        if (tag1 == tag2) and (tag1 == "O"): # O + O => False
            return False
        if (tag1 != tag2) and (tag1 != "O") and (tag2 != "O"): # X + Y => False
            return False
        return rep1.equals(rep2)
    else:
        half = int(n / 2)
        return is_mergeable(aListOfConsecutiveTokens[0:half+1], corpus, fingerprints) and is_mergeable(aListOfConsecutiveTokens[half:n], corpus, fingerprints)

In [170]:
def merge(depth, corpus, fingerprint):
    wordDf = corpus[corpus.word != "\n"].word
    nbOfWord = wordDf.shape[0]
    text = list(wordDf)
    X2, target2, tokens = [], [], []
    level, newToken = 1, True
    while level <= depth and newToken:
        i, newToken = 0, False
        limit = nbOfWord - level
        while i < limit:
            if is_mergeable(text[i:i+level+1], corpus, fingerprint):
                tokens.append(" ".join(text[i:i+level+1]))
                newToken = True
            i += 1
        print("level ", level, ":", set(tokens))
        level += 1
    
    X2, target2 = np.array(X2), np.array(target2)
    
    return X2, target2, set(tokens)

In [171]:
def shuffle(X, y):
    indices = [i for i in  range(X.shape[0])]
    np.random.shuffle(indices)
    return X[indices], y[indices]

In [172]:
en_corpus, en_nb_of_phrases = load_corpus(en_corpus_file, max_nb_of_phrases)

In [173]:
tagSet = en_corpus["ne-tag"].dropna().unique()
if BINARY:
    tagSet = ['NE', 'O']
tag2int = {j: i for i, j in enumerate(tagSet)}
int2tag = {i: j for i, j in enumerate(tagSet)}
print(tag2int)

{'O': 0, 'MISC': 1, 'PER': 2, 'LOC': 3, 'ORG': 4}


In [174]:
en_nb_of_phrases

210

In [175]:
en_corpus.describe()

,ne-tag,word
count,4170,4379
unique,5,904
top,O,the
freq,3779,313


In [176]:
en_corpus.head(10)

,ne-tag,word
0,O,The
1,O,Promise
2,O,of
3,O,the
4,MISC,Holy
5,MISC,Spirit
6,None,\n
7,O,In
8,O,the
9,O,first


In [177]:
for tag in tagSet:
    print("{0} % = {1} %".format(tag, np.round(en_corpus[en_corpus['ne-tag']==tag].shape[0] * 100 / en_corpus[en_corpus['ne-tag']!='\n'].shape[0], 2)))

O % = 86.3 %
MISC % = 2.4 %
PER % = 5.59 %
LOC % = 0.91 %
ORG % = 0.02 %


In [178]:
for tag in tagSet:
    print("{0} % = {1} %".format(tag, np.round(en_corpus[en_corpus['ne-tag']==tag].word.unique().shape[0] * 100 / en_corpus[en_corpus['ne-tag']!='\n'].word.unique().shape[0], 2)))

O % = 89.16 %
MISC % = 1.88 %
PER % = 8.96 %
LOC % = 1.99 %
ORG % = 0.11 %


In [179]:
en_corpus[en_corpus.word == "\n"].shape

(209, 2)

In [180]:
print("Nb of bi-phrases", en_nb_of_phrases)
en_fingerprints = corpus_fingerprint(en_corpus, en_nb_of_phrases)

Nb of bi-phrases 210


In [181]:
en_fingerprints.head(10)

,'And,'Let,'May,'The,'Why,(Now,(the,(which,120),Aaron,...,word,words,would,write,writing,written,wrote,years,you,your
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4170.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4170.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4170.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4170.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1390.0,0.0


In [182]:
en_fingerprints['you'].values.shape

(210,)

In [183]:
en_corpus[en_corpus.word != "\n"].shape

(4170, 2)

In [184]:
X2, target2, tokens = merge(max_depth, en_corpus, en_fingerprints)

In [185]:
if is_only_vocab:
    text = list(en_corpus[en_corpus.word != "\n"].word.unique())
else:
    text = list(en_corpus[en_corpus.word != "\n"].word)
en_vocab = pd.DataFrame({'text': text + list(tokens)})
en_vocab.describe()

,text
count,903
unique,903
top,fear
freq,1


In [186]:
if is_only_vocab:
    X = np.zeros((en_vocab.shape[0] * duplication, en_nb_of_phrases))
    target = np.zeros((en_vocab.shape[0] * duplication))
    p=0
    for i, row in en_vocab.iterrows():
        c = row.text
        for j in range(duplication):
            X[p] = en_fingerprints[c.split(" ")[0]]
            target[p] = tag2int[getTag(en_corpus[en_corpus.word == c.split(" ")[-1:][0]]['ne-tag'].iloc[0])]
            p+=1
    X, target = shuffle(X, target)
    print(X.shape, en_fingerprints.shape, target.shape)

(903, 210) (210, 903) (903,)


In [187]:
en_vocab[-20:]

,text
883,Eliud
884,Eleazar
885,Matthan
886,husband
887,fourteen
888,unwilling
889,shame
890,resolved
891,divorce
892,quietly


In [188]:
if not is_only_vocab:
    X, target = corpus2trainingdata(en_corpus[en_corpus.word != "\n"], en_fingerprints)

In [189]:
print(X.shape, target.shape)
if len(X.shape) == len(X2.shape):
    X = np.concatenate((X, X2))
    target = np.concatenate((target, target2))
    if shuffle:
        X, target = shuffle(X, target)
print(X.shape, target.shape)

(903, 210) (903,)
(903, 210) (903,)


In [190]:
y = target.copy()
y[0:100]
if not BINARY:
    y = np_utils.to_categorical(y, len(tagSet))
y.shape

(903, 5)

In [191]:
X_train, X_val, y_train, y_val = model_selection.train_test_split(X, y, test_size=0.33)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1])
X_val = X_val.reshape(X_val.shape[0], X_val.shape[1])
print("X_train.shape =", X_train.shape)
print("y_train.shape =", y_train.shape)
print("X_val.shape =", X_val.shape)
print("y_val.shape =", y_val.shape)

tTarget = np.array([np.argmax(yy) for yy in y_train])
vTarget = np.array([np.argmax(yy) for yy in y_val])

for tag in tagSet:
    print("{0} % in training data = {1} %".format(tag, np.round(tTarget[tTarget==tag2int[tag]].size * 100 / tTarget.shape[0], 2)))
    print("{0} % in validation data = {1} %".format(tag, np.round(vTarget[vTarget==tag2int[tag]].size * 100 / vTarget.shape[0], 2)))

X_train.shape = (605, 210)
y_train.shape = (605, 5)
X_val.shape = (298, 210)
y_val.shape = (298, 5)
O % in training data = 88.76 %
O % in validation data = 87.25 %
MISC % in training data = 1.32 %
MISC % in validation data = 1.01 %
PER % in training data = 7.77 %
PER % in validation data = 10.07 %
LOC % in training data = 2.15 %
LOC % in validation data = 1.34 %
ORG % in training data = 0.0 %
ORG % in validation data = 0.34 %


In [192]:
def create_model(input_dim, output_dim):
    model = Sequential()
    model.add(Dense(h1_size, input_dim=input_dim, activation='sigmoid', name="hidden1"))
    model.add(Dense(h2_size, activation='sigmoid', name="hidden2"))
    if BINARY:
        model.add(Dense(1, activation='sigmoid', name="outputlayer"))
        model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['binary_accuracy'])
    else:
        model.add(Dense(output_dim, activation='softmax', name="outputlayer"))
        model.compile(loss='categorical_crossentropy', optimizer="rmsprop", metrics=['accuracy'])
    model.summary()
    return model

In [193]:
def train_model(model, X_train, y_train, X_val, y_val, epochs=epochs):
    early_stop = EarlyStopping(patience=2, verbose=2) # stop learning if the error is the same between two consecutive epochs
    best_model_cp = ModelCheckpoint(best_model_file, save_best_only=True, verbose=1) # saved best model
    model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=epochs, verbose=0, shuffle=shuffle, callbacks=[best_model_cp, early_stop])
    best_model = keras.models.load_model(best_model_file) #loading the best model
    return best_model

In [194]:
def predict(model, X, y, binary=BINARY):
    if BINARY:
        y_pred = np.round(model.predict(X))
        y_true = y
    else:
        predictions = model.predict(X)
        y_pred = np.array([np.argmax(p) for p in predictions])
        y_true = np.array([np.argmax(t) for t in y ])
    return y_true, y_pred

In [195]:
def model_performance(y_true, y_pred):
    return P_R_F1(y_pred, y_true, tag2int['O']) #precision, recall, f1-score

In [196]:
def model_performace_by_tag(y_true, y_pred, tag):
    p, r, f1 = 0, 0, 0
    
    eq = y_pred[y_pred==y_true]
    correctly_pred = eq[eq==tag].size
    try:
        p = np.round(100 * correctly_pred / y_pred[y_pred==tag].size, 2)
    except ZeroDivisionError:
        pass
    
    try:
        r = np.round(100 * correctly_pred / y_true[y_true==tag].size, 2)
    except ZeroDivisionError:
        pass
    
    try:
        f1 = np.round(2 * r * p / (r + p), 2)
    except ZeroDivisionError:
        pass
    
    return p, r, f1

In [197]:
ewo_corpus, ewo_nb_of_phrases = load_corpus(ewo_corpus_file, max_nb_of_phrases)

In [198]:
ewo_nb_of_phrases

210

In [199]:
for tag in tagSet:
    print("{0} % = {1} %".format(tag, np.round(ewo_corpus[ewo_corpus['ne-tag']==tag].shape[0] * 100 / ewo_corpus[ewo_corpus['ne-tag']!='\n'].shape[0], 2)))

O % = 84.15 %
MISC % = 2.54 %
PER % = 6.69 %
LOC % = 1.03 %
ORG % = 0.05 %


In [200]:
for tag in tagSet:
    print("{0} % = {1} %".format(tag, np.round(ewo_corpus[ewo_corpus['ne-tag']==tag].word.unique().shape[0] * 100 / ewo_corpus[ewo_corpus['ne-tag']!='\n'].word.unique().shape[0], 2)))

O % = 89.94 %
MISC % = 1.17 %
PER % = 8.3 %
LOC % = 1.86 %
ORG % = 0.2 %


In [201]:
ewo_corpus.describe()

,ne-tag,word
count,3570,3779
unique,5,1024
top,O,\n
freq,3180,209


In [202]:
ewo_corpus.head()

,ne-tag,word
0,MISC,Mfufub
1,MISC,Nsisim
2,O,ayi
3,O,sò
4,None,\n


In [203]:
ewo_fingerprints = corpus_fingerprint(ewo_corpus, en_nb_of_phrases)

In [204]:
X2, target2, tokens = merge(max_depth, ewo_corpus, ewo_fingerprints)

In [205]:
if is_only_vocab:
    text = list(ewo_corpus[ewo_corpus.word != "\n"].word.unique())
else:
    text = list(ewo_corpus[ewo_corpus.word != "\n"].word)
ewo_vocab = pd.DataFrame({"text":text + list(tokens)})

In [206]:
if is_only_vocab:
    X_ewo = np.zeros((ewo_vocab.shape[0] * duplication, en_nb_of_phrases))
    ewo_target = np.zeros((ewo_vocab.shape[0] * duplication))
    p=0
    for i, row in ewo_vocab.iterrows():
        c = row.text
        for j in range(duplication):
            X_ewo[p] = ewo_fingerprints[c.split(" ")[0]]
            ewo_target[p] = tag2int[getTag(ewo_corpus[ewo_corpus.word == c.split(" ")[-1:][0]]['ne-tag'].iloc[0])]
            p+=1
    X_ewo, ewo_target = shuffle(X_ewo, ewo_target)

In [207]:
ewo_vocab[-20:]

,text
1003,nlo
1004,obë
1005,mbara
1006,yabyali
1007,dzili
1008,yasò
1009,oyolëge
1010,kode
1011,dili
1012,atoban


In [208]:
if not is_only_vocab:
    X_ewo, ewo_target = corpus2trainingdata(ewo_corpus[ewo_corpus.word != "\n"], ewo_fingerprints)

In [209]:
print(X_ewo.shape, ewo_target.shape)
if len(X_ewo.shape) == len(X2.shape):
    X_ewo = np.concatenate((X_ewo, X2))
    ewo_target = np.concatenate((ewo_target, target2))
    if shuffle:
        X_ewo, ewo_target = shuffle(X_ewo, ewo_target)
print(X_ewo.shape, ewo_target.shape)

(1023, 210) (1023,)
(1023, 210) (1023,)


In [210]:
y_ewo = ewo_target.copy()
print(y_ewo.shape, len(ewo_vocab))

(1023,) 1023


In [211]:
X_ewo.shape

(1023, 210)

In [212]:
y_ewo = ewo_target.copy()
y_ewo[:20]
if not BINARY:
    y_ewo = np_utils.to_categorical(y_ewo)

In [213]:
X_ewo = X_ewo.reshape((X_ewo.shape[0], en_nb_of_phrases))

In [214]:
def algoEval(X_train, y_train, X_val, y_val, X_ewo, y_ewo, epochs=epochs, model=None):
    test_precision, train_precision, ewo_precision = [], [], []
    test_recall, train_recall, ewo_recall = [], [], []
    test_fscore, train_fscore, ewo_fscore = [], [], []
    
    test_result_by_tag = {}
    train_result_by_tag = {}
    ewo_result_by_tag = {}
    for t in tagSet:
        f1_key = "F1-"+t
        p_key = "P-"+t
        r_key = "R-"+t
        train_result_by_tag[f1_key], train_result_by_tag[p_key], train_result_by_tag[r_key] = [], [], []
        test_result_by_tag[f1_key], test_result_by_tag[p_key], test_result_by_tag[r_key] = [], [], []
        ewo_result_by_tag[f1_key], ewo_result_by_tag[p_key], ewo_result_by_tag[r_key] = [], [], []

    m = train_model(model, X_train, y_train, X_val, y_val, epochs=epochs)
        
    y_true, y_pred = predict(m, X_train, y_train)
    p_train, r_train, f1_train = model_performance(y_true, y_pred)
        
    y_true_val, y_pred_val = predict(m, X_val, y_val)
    p_val, r_val, f1_val = model_performance(y_true_val, y_pred_val)
        
    y_true_ewo, y_pred_ewo = predict(m, X_ewo, y_ewo) 
    p_ewo, r_ewo, f1_ewo = model_performance(y_true_ewo, y_pred_ewo)
        
    for t in range(len(int2tag)):
        f1_key = "F1-" + int2tag[t]
        p_key = "P-" + int2tag[t]
        r_key = "R-" + int2tag[t]
            
        p, r, f1 = model_performace_by_tag(y_true, y_pred, t)
        train_result_by_tag[p_key].append(p)
        train_result_by_tag[r_key].append(r)
        train_result_by_tag[f1_key].append(f1)
            
        p, r, f1 = model_performace_by_tag(y_true_val, y_pred_val, t)
        test_result_by_tag[p_key].append(p)
        test_result_by_tag[r_key].append(r)
        test_result_by_tag[f1_key].append(f1)
            
        p, r, f1 = model_performace_by_tag(y_true_ewo, y_pred_ewo, t)
        ewo_result_by_tag[p_key].append(p)
        ewo_result_by_tag[r_key].append(r)
        ewo_result_by_tag[f1_key].append(f1)
                
    test_precision.append(p_val)
    train_precision.append(p_train)
    ewo_precision.append(p_ewo)
        
    test_recall.append(r_val)
    train_recall.append(r_train)
    ewo_recall.append(r_ewo)
        
    test_fscore.append(f1_val)
    train_fscore.append(f1_train)
    ewo_fscore.append(f1_ewo)
    return pd.DataFrame({
        'P_test': test_precision, 
        'P_train': train_precision, 
        'P_ewo': ewo_precision, 'R_test': test_recall, 'R_train': train_recall, 
        'R_ewo': ewo_recall, 'F1-test': test_fscore, 'F1-train': train_fscore, 'F1-ewo': ewo_fscore}), pd.DataFrame(train_result_by_tag), pd.DataFrame(test_result_by_tag), pd.DataFrame(ewo_result_by_tag)

In [215]:
# model = create_model(X.shape[1], len(tagSet))
# resultEval, train_by_tag, test_by_tag, ewo_by_tag = algoEval(X_train, y_train, X_val, y_val, X_ewo, y_ewo, model=model)

In [216]:
# resultEval

In [217]:
# train_by_tag

In [218]:
# test_by_tag

In [219]:
# ewo_by_tag

In [220]:
# resultEval.mean()

In [221]:
# resultEval.std()

In [222]:
def algoCrossVal(X, y, X_ewo, y_ewo, k = 10, repeat=1): 
    block_size = int(X.shape[0] / k)   
    output = None
    model = None
    train_by_tags, test_by_tags, ewo_by_tags = None, None, None
    for it in range(repeat):
        print("AlgoCrossValIter -", it+1)
        model = create_model(X.shape[1], len(tagSet))
        results = None
        train_by_tagsTmp, test_by_tagsTmp, ewo_by_tagsTmp = None, None, None
        for i in range(k):
            X_val, y_val = X[i*block_size:i*block_size+block_size], y[i*block_size:i*block_size+block_size]
            X_train = np.concatenate((X[0:i*block_size], X[i*block_size+block_size:]))
            y_train = np.concatenate((y[0:i*block_size], y[i*block_size+block_size:]))

            X_train = X_train.reshape(X_train.shape[0], X_train.shape[1])
            X_val = X_val.reshape(X_val.shape[0], X_val.shape[1])

            result, train_by_tag, test_by_tag, ewo_by_tag = algoEval(X_train, y_train, X_val, y_val, X_ewo, y_ewo, model=model)
            if results is None:
                results = result.copy()
                train_by_tagsTmp, test_by_tagsTmp, ewo_by_tagsTmp = train_by_tag.copy(), test_by_tag.copy(), ewo_by_tag.copy()
            else:
                results = pd.concat([results, result], ignore_index=True)
                train_by_tagsTmp = pd.concat([train_by_tagsTmp, train_by_tag], ignore_index=True)
                test_by_tagsTmp = pd.concat([test_by_tagsTmp, test_by_tag], ignore_index=True)
                ewo_by_tagsTmp = pd.concat([ewo_by_tagsTmp, ewo_by_tag], ignore_index=True)
        
        if output is None:
            output = results.mean(axis=0).to_frame()
            train_by_tags = train_by_tagsTmp.mean(axis=0).to_frame()
            test_by_tags = test_by_tagsTmp.mean(axis=0).to_frame()
            ewo_by_tags = ewo_by_tagsTmp.mean(axis=0).to_frame()
        else:
            output = pd.concat([output, results.mean(axis=0).to_frame()], axis=1)
            train_by_tags = pd.concat([train_by_tags, train_by_tagsTmp.mean(axis=0).to_frame()], axis=1)
            test_by_tags = pd.concat([test_by_tags, test_by_tagsTmp.mean(axis=0).to_frame()], axis=1)
            ewo_by_tags = pd.concat([ewo_by_tags, ewo_by_tagsTmp.mean(axis=0).to_frame()], axis=1)

    return output, train_by_tags, test_by_tags, ewo_by_tags, model

In [223]:
resultCrossVal, trainByTagResult, testByTagResult, ewoByTagResult, model = algoCrossVal(X, y, X_ewo, y_ewo, repeat=10)

AlgoCrossValIter - 1
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hidden1 (Dense)              (None, 640)               135040    
_________________________________________________________________
hidden2 (Dense)              (None, 160)               102560    
_________________________________________________________________
outputlayer (Dense)          (None, 5)                 805       
Total params: 238,405
Trainable params: 238,405
Non-trainable params: 0
_________________________________________________________________
Epoch 00001: val_loss improved from inf to 0.54100, saving model to best-model-conll.hdfs
Epoch 00002: val_loss improved from 0.54100 to 0.43521, saving model to best-model-conll.hdfs
Epoch 00003: val_loss improved from 0.43521 to 0.40635, saving model to best-model-conll.hdfs
Epoch 00004: val_loss improved from 0.40635 to 0.38309, saving model to best-model-conll.hdfs
Epoch 0

Epoch 00009: val_loss did not improve
Epoch 00009: early stopping
Epoch 00001: val_loss improved from inf to 0.18834, saving model to best-model-conll.hdfs
Epoch 00002: val_loss did not improve
Epoch 00003: val_loss did not improve
Epoch 00003: early stopping
Epoch 00001: val_loss improved from inf to 0.04713, saving model to best-model-conll.hdfs
Epoch 00002: val_loss did not improve
Epoch 00003: val_loss did not improve
Epoch 00003: early stopping
Epoch 00001: val_loss improved from inf to 0.03942, saving model to best-model-conll.hdfs
Epoch 00002: val_loss did not improve
Epoch 00003: val_loss did not improve
Epoch 00003: early stopping
Epoch 00001: val_loss improved from inf to 0.03275, saving model to best-model-conll.hdfs
Epoch 00002: val_loss did not improve
Epoch 00003: val_loss did not improve
Epoch 00003: early stopping
Epoch 00001: val_loss improved from inf to 0.11846, saving model to best-model-conll.hdfs
Epoch 00002: val_loss did not improve
Epoch 00003: val_loss improved

Epoch 00002: val_loss did not improve
Epoch 00003: val_loss did not improve
Epoch 00003: early stopping
Epoch 00001: val_loss improved from inf to 0.10322, saving model to best-model-conll.hdfs
Epoch 00002: val_loss did not improve
Epoch 00003: val_loss did not improve
Epoch 00003: early stopping
Epoch 00001: val_loss improved from inf to 0.08226, saving model to best-model-conll.hdfs
Epoch 00002: val_loss improved from 0.08226 to 0.05886, saving model to best-model-conll.hdfs
Epoch 00003: val_loss did not improve
Epoch 00004: val_loss did not improve
Epoch 00004: early stopping
Epoch 00001: val_loss improved from inf to 0.10928, saving model to best-model-conll.hdfs
Epoch 00002: val_loss improved from 0.10928 to 0.10852, saving model to best-model-conll.hdfs
Epoch 00003: val_loss did not improve
Epoch 00004: val_loss did not improve
Epoch 00004: early stopping
AlgoCrossValIter - 6
_________________________________________________________________
Layer (type)                 Output Sha

Epoch 00004: val_loss did not improve
Epoch 00005: val_loss did not improve
Epoch 00005: early stopping
AlgoCrossValIter - 8
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hidden1 (Dense)              (None, 640)               135040    
_________________________________________________________________
hidden2 (Dense)              (None, 160)               102560    
_________________________________________________________________
outputlayer (Dense)          (None, 5)                 805       
Total params: 238,405
Trainable params: 238,405
Non-trainable params: 0
_________________________________________________________________
Epoch 00001: val_loss improved from inf to 0.53078, saving model to best-model-conll.hdfs
Epoch 00002: val_loss improved from 0.53078 to 0.47619, saving model to best-model-conll.hdfs
Epoch 00003: val_loss improved from 0.47619 to 0.42615, saving model to best-model-conll.hd

Epoch 00003: val_loss did not improve
Epoch 00004: val_loss did not improve
Epoch 00004: early stopping
Epoch 00001: val_loss improved from inf to 0.06010, saving model to best-model-conll.hdfs
Epoch 00002: val_loss did not improve
Epoch 00003: val_loss improved from 0.06010 to 0.05937, saving model to best-model-conll.hdfs
Epoch 00004: val_loss did not improve
Epoch 00005: val_loss did not improve
Epoch 00005: early stopping
Epoch 00001: val_loss improved from inf to 0.05778, saving model to best-model-conll.hdfs
Epoch 00002: val_loss did not improve
Epoch 00003: val_loss did not improve
Epoch 00003: early stopping
Epoch 00001: val_loss improved from inf to 0.04652, saving model to best-model-conll.hdfs
Epoch 00002: val_loss improved from 0.04652 to 0.03045, saving model to best-model-conll.hdfs
Epoch 00003: val_loss did not improve
Epoch 00004: val_loss did not improve
Epoch 00004: early stopping
Epoch 00001: val_loss improved from inf to 0.08720, saving model to best-model-conll.hdf

In [224]:
resultCrossVal.to_csv("results/merge-{0}.csv".format(max_depth))
resultCrossVal

,0,0,0,0,0,0,0,0,0,0
F1-ewo,70.931,68.972,71.205,65.211,71.920,69.447,69.533,69.780,70.942,68.314
F1-test,78.094,80.950,82.098,77.588,82.185,77.646,81.185,81.531,80.715,77.030
F1-train,83.808,83.480,85.419,78.398,86.461,83.548,84.376,81.526,84.897,81.605
P_ewo,81.039,78.039,76.718,83.650,80.761,79.206,78.462,79.611,78.228,77.845
P_test,85.411,87.977,84.065,90.253,85.809,82.646,83.042,87.224,83.313,87.324
P_train,89.314,87.979,87.343,90.533,89.546,89.607,88.857,87.167,87.777,87.421
R_ewo,64.630,63.334,67.223,55.370,65.093,63.056,63.981,63.889,66.205,62.872
R_test,76.921,78.993,83.800,71.120,81.843,77.070,80.588,80.948,80.301,74.889
R_train,80.183,80.546,84.010,71.184,83.789,79.025,81.784,78.041,82.941,78.946


In [225]:
resultCrossVal.mean(axis=1).to_frame()

,0
F1-ewo,69.6255
F1-test,79.9022
F1-train,83.3518
P_ewo,79.3559
P_test,85.7064
P_train,88.5544
R_ewo,63.5653
R_test,78.6473
R_train,80.0449


In [226]:
resultCrossVal.std(axis=1).to_frame()

,0
F1-ewo,1.911842
F1-test,2.056288
F1-train,2.322956
P_ewo,2.010549
P_test,2.488470
P_train,1.168430
R_ewo,3.198061
R_test,3.725731
R_train,3.744645


In [227]:
trainByTagResult.to_csv("results/train-by-tag-merge-{0}.csv".format(max_depth))
trainByTagResult

,0,0,0,0,0,0,0,0,0,0
F1-LOC,70.725556,73.768889,74.913,67.653333,78.771,70.567000,74.488000,72.430000,70.292,74.930000
F1-MISC,76.858000,72.330000,76.801,70.236000,74.949,71.784000,78.661000,71.054444,76.148,72.724000
F1-O,98.029000,97.968000,98.147,97.616000,98.263,97.996000,98.052000,97.819000,98.077,97.870000
F1-ORG,NaN,44.446667,60.002,33.335000,58.335,44.446667,44.446667,50.002500,33.335,44.446667
F1-PER,88.074000,87.653000,88.880,82.577000,89.608,87.493000,87.302000,86.277000,88.562,85.624000
P-LOC,80.088000,80.750000,79.379,79.950000,89.923,88.647000,84.809000,71.883000,84.856,73.073000
P-MISC,93.460000,93.639000,98.889,94.750000,98.889,96.528000,94.960000,87.639000,96.528,95.051000
P-O,97.456000,97.515000,97.917,96.359000,97.861,97.274000,97.639000,97.215000,97.759,97.329000
P-ORG,0.000000,10.000000,25.000,5.000000,20.000,10.000000,10.000000,15.000000,5.000,10.000000
P-PER,89.658000,87.908000,88.384,90.948000,89.066,89.668000,89.617000,88.798000,87.946,88.655000


In [228]:
trainByTagResult.mean(axis=1).to_frame()

,0
F1-LOC,72.853878
F1-MISC,74.154544
F1-O,97.983700
F1-ORG,45.866241
F1-PER,87.205000
P-LOC,81.335800
P-MISC,95.033300
P-O,97.432400
P-ORG,11.000000
P-PER,89.064800


In [229]:
trainByTagResult.std(axis=1).to_frame()

,0
F1-LOC,3.173122
F1-MISC,2.890330
F1-O,0.181277
F1-ORG,9.317477
F1-PER,2.006900
P-LOC,5.934153
P-MISC,3.221147
P-O,0.448516
P-ORG,7.378648
P-PER,0.934500


In [230]:
testByTagResult.to_csv("results/test-by-tag-merge-{0}.csv".format(max_depth))
testByTagResult

,0,0,0,0,0,0,0,0,0,0
F1-LOC,86.111667,84.285714,81.852222,66.666667,91.6675,74.445,82.976250,85.238571,80.83375,86.666667
F1-MISC,45.833750,54.167500,50.000000,54.167500,56.2500,50.000,51.852222,56.250000,50.00000,56.250000
F1-O,97.299000,97.621000,97.519000,97.511000,97.6670,97.163,97.590000,97.603000,97.40500,97.390000
F1-ORG,10.000000,0.000000,0.000000,0.000000,0.0000,0.000,0.000000,0.000000,0.00000,0.000000
F1-PER,84.768000,87.511000,87.407000,83.176000,86.3090,84.212,85.673000,88.415000,86.10300,82.336000
P-LOC,56.667000,65.000000,83.334000,50.000000,76.6670,51.667,68.334000,63.334000,71.66700,55.000000
P-MISC,33.333000,45.000000,38.333000,45.000000,45.0000,38.333,48.333000,45.000000,38.33300,45.000000
P-O,96.730000,96.844000,97.526000,96.124000,97.2890,96.831,97.378000,97.196000,97.14500,96.418000
P-ORG,10.000000,0.000000,0.000000,0.000000,0.0000,0.000,0.000000,0.000000,0.00000,0.000000
P-PER,88.317000,89.845000,86.717000,90.333000,87.1460,84.820,86.241000,90.480000,86.16700,89.230000


In [231]:
testByTagResult.mean(axis=1).to_frame()

,0
F1-LOC,82.074401
F1-MISC,52.477097
F1-O,97.476800
F1-ORG,1.000000
F1-PER,85.591000
P-LOC,64.167000
P-MISC,42.166500
P-O,96.948100
P-ORG,1.000000
P-PER,87.929600


In [232]:
testByTagResult.std(axis=1).to_frame()

,0
F1-LOC,7.010514
F1-MISC,3.517447
F1-O,0.160358
F1-ORG,3.162278
F1-PER,1.964967
P-LOC,11.034609
P-MISC,4.717429
P-O,0.443156
P-ORG,3.162278
P-PER,1.986144


In [233]:
ewoByTagResult.to_csv("results/ewo-by-tag-merge-{0}.csv".format(max_depth))

In [234]:
ewoByTagResult = pd.read_csv("results/ewo-by-tag-merge-{0}.csv".format(2), index_col=0)
ewoByTagResult

,0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9
F1-LOC,NaN,NaN,NaN,NaN,NaN,17.380000,NaN,NaN,NaN,17.380000
F1-MISC,46.15000,46.150000,46.150000,46.1500,43.6825,43.682500,46.15000,46.15000,46.150,43.591111
F1-O,94.43100,95.075000,94.737000,94.9000,94.9640,95.227000,94.91800,94.86800,95.157,95.299000
F1-ORG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
F1-PER,8.57625,47.702857,28.391429,32.1575,34.5975,39.691111,34.62875,31.67875,42.050,41.984444
P-LOC,0.00000,0.000000,0.000000,0.0000,0.0000,60.000000,0.00000,0.00000,0.000,60.000000
P-MISC,70.00000,60.000000,60.000000,70.0000,65.0000,65.000000,60.00000,60.00000,80.000,72.500000
P-O,89.54000,91.146000,90.138000,90.5000,90.7090,91.122000,90.53800,90.38400,91.105,91.426000
P-ORG,0.00000,0.000000,0.000000,0.0000,0.0000,0.000000,0.00000,0.00000,0.000,0.000000
P-PER,53.63800,51.554000,57.193000,63.3810,66.7380,77.824000,66.39100,68.14200,66.391,75.073000


In [235]:
ewoByTagResult.mean(axis=1).to_frame()

,0
F1-LOC,17.380000
F1-MISC,45.400611
F1-O,94.957600
F1-ORG,NaN
F1-PER,34.145859
P-LOC,12.000000
P-MISC,66.250000
P-O,90.660800
P-ORG,0.000000
P-PER,64.632500


In [236]:
ewoByTagResult.std(axis=1).to_frame()

,0
F1-LOC,0.000000
F1-MISC,1.206887
F1-O,0.254209
F1-ORG,NaN
F1-PER,10.728242
P-LOC,25.298221
P-MISC,6.795628
P-O,0.565721
P-ORG,0.000000
P-PER,8.529639


In [237]:
columns = en_fingerprints.columns

print("Pred", "Real", "Freq", "Word", sep="\t")
for c in columns:
    prediction = model.predict(en_fingerprints[c].values.reshape((1, 1, 210)))
    pred_tag = int2tag[np.argmax(prediction)]
    real_tag = en_corpus[en_corpus.word == c].iloc[0]['ne-tag']
    
    if pred_tag != real_tag:
        print(pred_tag, real_tag, en_fingerprints[c].max(), c, sep="\t")

Pred	Real	Freq	Word


ValueError: Error when checking : expected hidden1_input to have 2 dimensions, but got array with shape (1, 1, 210)

In [ ]:
en_corpus[en_corpus.word != "\n"].shape